In [33]:
from nhlpy import NHLClient
import pandas as pd

In [4]:
client = NHLClient()

In [2]:
game_id = '2022020023'

In [30]:
def second_diff(time1, time2):
    minutes1 = int(time1[0:2])
    minutes2 = int(time2[0:2])
    seconds1 = int(time1[3:5])
    seconds2 = int(time2[3:5])
    return abs((minutes2 * 60 + seconds2) - (minutes1 * 60 + seconds1))

In [101]:
rows = []
home_id = client.game_center.play_by_play(game_id=game_id)['homeTeam']['id']
away_id = client.game_center.play_by_play(game_id=game_id)['awayTeam']['id']
pbp = client.game_center.play_by_play(game_id=game_id)['plays']
idx = -1

for play in pbp:
    idx += 1
    if play['typeDescKey'] not in ['missed-shot', 'goal', 'shot-on-goal']:
        continue
    home = 0
    away = 0
    rebound = 0
    rush = 0
    try:
        if home_id == play['details']['eventOwnerTeamId']:
            home = 1
        else:
            away = 1

        if (home == 1 and play['situationCode'][0] == '0') or (away == 1 and play['situationCode'][3] == '0'):
            continue

        if home == 1:
            team_id = home_id
        else:
            team_id = away_id
            
        time_diff = second_diff(play['timeInPeriod'], pbp[idx - 1]['timeInPeriod'])

        if pbp[idx - 1]['typeDescKey'] == 'blocked-shot' and time_diff <= 2:
            rebound = 1
        
        if (pbp[idx-1]['typeDescKey'] in ['missed-shot', 'shot-on-goal']) and time_diff <= 3:
            rebound = 1

        if (pbp[idx-1]['typeDescKey'] in ['takeaway', 'giveaway']) and time_diff <= 4 and pbp[idx-1]['details']['zoneCode'] in ['N', 'D']:
            rush = 1

        assist1 = 0
        assist2 = 0
        home_skaters = play['situationCode'][2]
        away_skaters = play['situationCode'][1]
        shot_class = play['typeDescKey']
        x_coord = abs(play['details']['xCoord'])
        y_coord = play['details']['yCoord']
        shot_type = play['details']['shotType']
        shooter = None
        last_play = pbp[idx-1]['typeDescKey']
        if shot_class == 'goal':
            shooter = play['details']['scoringPlayerId']
            if 'assist1PlayerId' in play['details']:
                assist1 = play['details']['assist1PlayerId']
            if 'assist2PlayerId' in play['details']:
                assist2 = play['details']['assist2PlayerId']
        else:
            shooter = play['details']['shootingPlayerId']

        rows.append([game_id, team_id, home, last_play, rebound, rush, home_skaters, away_skaters, x_coord, y_coord, shooter, assist1, assist2, shot_type, shot_class])
    except:
        continue

In [102]:
header = ['game_id', 'team_id', 'home', 'last_play', 'rebound', 'rush', 'home_skaters', 'away_skaters', 'x_coord', 'y_coord', 'shooter', 'assist1', 'assist2', 'shot_type', 'shot_class']

In [103]:
len(before)

104

In [ ]:
rows

[['2022020023',
  52,
  1,
  'giveaway',
  0,
  1,
  '5',
  '5',
  77,
  6,
  8480289,
  0,
  0,
  'wrist',
  'shot-on-goal'],
 ['2022020023',
  52,
  1,
  'hit',
  0,
  0,
  '5',
  '5',
  30,
  30,
  8480145,
  0,
  0,
  'slap',
  'shot-on-goal'],
 ['2022020023',
  3,
  0,
  'shot-on-goal',
  0,
  0,
  '5',
  '5',
  35,
  30,
  8479333,
  0,
  0,
  'wrist',
  'shot-on-goal'],
 ['2022020023',
  52,
  1,
  'hit',
  0,
  0,
  '5',
  '5',
  41,
  14,
  8471218,
  0,
  0,
  'wrist',
  'shot-on-goal'],
 ['2022020023',
  52,
  1,
  'hit',
  0,
  0,
  '5',
  '5',
  46,
  17,
  8471218,
  0,
  0,
  'wrist',
  'shot-on-goal'],
 ['2022020023',
  52,
  1,
  'hit',
  0,
  0,
  '5',
  '5',
  81,
  -27,
  8477940,
  0,
  0,
  'wrist',
  'missed-shot'],
 ['2022020023',
  52,
  1,
  'missed-shot',
  0,
  0,
  '5',
  '5',
  34,
  30,
  8480145,
  0,
  0,
  'wrist',
  'shot-on-goal'],
 ['2022020023',
  3,
  0,
  'faceoff',
  0,
  0,
  '4',
  '5',
  69,
  2,
  8476389,
  0,
  0,
  'deflected',
  'shot-on

In [105]:
df = pd.DataFrame(rows, columns=header)
df.head()

,game_id,team_id,home,last_play,rebound,rush,home_skaters,away_skaters,x_coord,y_coord,shooter,assist1,assist2,shot_type,shot_class
0,2022020023,52,1,giveaway,0,1,5,5,77,6,8480289,0,0,wrist,shot-on-goal
1,2022020023,52,1,hit,0,0,5,5,30,30,8480145,0,0,slap,shot-on-goal
2,2022020023,3,0,shot-on-goal,0,0,5,5,35,30,8479333,0,0,wrist,shot-on-goal
3,2022020023,52,1,hit,0,0,5,5,41,14,8471218,0,0,wrist,shot-on-goal
4,2022020023,52,1,hit,0,0,5,5,46,17,8471218,0,0,wrist,shot-on-goal


In [106]:
df['shot_class'].value_counts()

shot_class
shot-on-goal    70
missed-shot     29
goal             4
Name: count, dtype: int64